Following figure is a typical Deep learning model [1] The Convolution part does the feature extraction from the input and the part with dense layers does the classification part. In our case we have extracted features. So, our model only consists of dense layers.
<img src="cnn.jpg" width="400" height="50">

Importing panda and numpy library to load data from the xlsx file into a data frame and extract features out of it.

In [1]:
import pandas as pd
import numpy as np

Next loading data into a data frame `df` and then displaying first few rows.

In [2]:
df =pd.read_excel("IT_3.xlsx")
df.head()

,ID,target,Gender,EngineHP,credit_history,Years_Experience,annual_claims,Marital_Status,Vehical_type,Miles_driven_annually,size_of_family,Age_bucket,EngineHP_bucket,Years_Experience_bucket,Miles_driven_annually_bucket,credit_history_bucket,State
0,1,1,F,522,656,1,0,Married,Car,14749.0,5,<18,>350,<3,<15k,Fair,IL
1,2,1,F,691,704,16,0,Married,Car,15389.0,6,28-34,>350,15-30,15k-25k,Good,NJ
2,3,1,M,133,691,15,0,Married,Van,9956.0,3,>40,90-160,15-30,<15k,Good,CT
3,4,1,M,146,720,9,0,Married,Van,77323.0,3,18-27,90-160,9-14',>25k,Good,CT
4,5,1,M,128,771,33,1,Married,Van,14183.0,4,>40,90-160,>30,<15k,Very Good,WY


In [ ]:
df.shape

In [ ]:
df.columns

We do not need `ID` column

In [3]:
df = df.drop(['ID'], axis=1)

In [ ]:
df.columns

Only selecting the columns which are going to use as feature

In [64]:
features = df[['Vehical_type',  'EngineHP_bucket', 'Years_Experience_bucket', 'Miles_driven_annually_bucket', 'credit_history_bucket', 'State']]

In [ ]:
features.columns

Next we are doing one hot encoding by using `pandas` function `get_dummies`, that means every value in a categorical type column will be turned into a column and the value will be replace by 1 and 0. 

| ID | car | millage |
| --- | --- | --- |
| 1 | Mercedes | 2000 |
| 2 | Mercedes | 50000 |
| 3 | Honda | 3000 |
| 4 | VW | 50 |

The above table will be turn into as following table.

| ID | car Mercedes | car Honda| car VW | millage |
| --- | --- | --- | --- | --- |
| 1 | 1 | 0 | 0 | 2000 |
| 2 | 1 | 0 | 0 | 50000 |
| 3 | 0 | 1 | 0 | 3000 |
| 4 | 0 | 0 | 1 | 50 |

In [65]:
features = pd.get_dummies(features)

In [ ]:
features.columns

In [66]:
features = features.to_numpy(copy = True)
features

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=uint8)

Using `target` columns for labels

In [7]:
labels = df[["target"]]
labels = labels.to_numpy(copy = True)
labels

array([[1],
       [1],
       [1],
       ...,
       [0],
       [1],
       [1]], dtype=int64)

Using `scikit-learn` library and `train_test_split` function to divide our data for training the model and testing the trained model.

In [67]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

Importing necessary libraries for defining deep learning model, layers and training and testing the model

In [9]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout

In [ ]:
len(features[0])

In [ ]:
len(features)

We create a model with few dense and dropout layers. For dense layer the activation function is `relu` and the last layer it is `sigmoid`. The first layer is input layer with input dimension is equal to the number of feature value. We print out the model summary at the end of defining the model.  

In [68]:
model = Sequential()
model.add(Dense(100, activation="relu", input_dim = len(features[0])))
model.add(Dropout(0.2))
model.add(Dense(70, activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(100, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 100)               7200      
_________________________________________________________________
dropout_16 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 70)                7070      
_________________________________________________________________
dropout_17 (Dropout)         (None, 70)                0         
_________________________________________________________________
dense_22 (Dense)             (None, 100)               7100      
_________________________________________________________________
dropout_18 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 20)               

To optimize the training process we use `adam` optimizer function, `binary_crossentropy` calculation for the loss calculation and then `accuracy` as a metrics. So, the model will try to adjust it weight and bias value to achieve higher possible accuracy with lower possible loss and in a optimized way.

In [69]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"] )

We have defined model and also define the compilation techniques. Now we shall run the actual training with the training data. It is better to use a portion of data for the validation of the training process to see whether with the data we are having under fitting or over fitting problem. To store the logs and to generate the diagram to compare the loss and accuracy curve of training and validation during training process we first need to use `tensorboard` library. Then we shall use `fit` function to run the actual training.  

In [70]:
from keras.callbacks import TensorBoard
from time import time
log_dir='logs/{}'.format(time())
tensorboard = TensorBoard(log_dir=log_dir)

In [71]:
#batch_size means how many number will take for fit
model.fit(train_features,train_labels,batch_size = 50, epochs= 50, validation_split =0.2, shuffle = True, callbacks=[tensorboard])

Epoch 1/50
363/363 [==============================] - 1s 3ms/step - loss: 0.6264 - accuracy: 0.6894 - val_loss: 0.6174 - val_accuracy: 0.7019
Epoch 2/50
363/363 [==============================] - 0s 1ms/step - loss: 0.6105 - accuracy: 0.7074 - val_loss: 0.6119 - val_accuracy: 0.7019
Epoch 3/50
363/363 [==============================] - 0s 1ms/step - loss: 0.6077 - accuracy: 0.7077 - val_loss: 0.6104 - val_accuracy: 0.7019
Epoch 4/50
363/363 [==============================] - 0s 1ms/step - loss: 0.6021 - accuracy: 0.7120 - val_loss: 0.6108 - val_accuracy: 0.7019
Epoch 5/50
363/363 [==============================] - 0s 1ms/step - loss: 0.6058 - accuracy: 0.7075 - val_loss: 0.6098 - val_accuracy: 0.7019
Epoch 6/50
363/363 [==============================] - 0s 1ms/step - loss: 0.6024 - accuracy: 0.7086 - val_loss: 0.6095 - val_accuracy: 0.7019
Epoch 7/50
363/363 [==============================] - 0s 1ms/step - loss: 0.6085 - accuracy: 0.7020 - val_loss: 0.6093 - val_accuracy: 0.7019
Epoch 

Now we plot loss and accuracy curve using the `tensorboard` to see whether there is under fitting or over fitting.

In [13]:
%load_ext tensorboard

In [74]:
%tensorboard --logdir {log_dir} --host localhost --port 8088

Reusing TensorBoard on port 8088 (pid 9924), started 0:00:59 ago. (Use '!kill 9924' to kill it.)

Finally, we evaluate the trained model with the test data.

In [72]:
loss,accuracy = model.evaluate(test_features,test_labels,batch_size = 10)

756/756 [==============================] - 1s 575us/step - loss: 0.6827 - accuracy: 0.6868


Printing loss and accuracy value as a final evaluation of our model to predict the `target` column that gives the indication whether the insurance holder is going to make a claim or not.

In [ ]:
print("loss: {}, accuracy: {}".format(loss,accuracy))

References:
1. Daniel Weimer, Bernd Scholz-Reiter, Moshe Shpitalni,
Design of deep convolutional neural network architectures for automated feature extraction in industrial inspection,
CIRP Annals,
Volume 65, Issue 1,
2016,
Pages 417-420,
ISSN 0007-8506,